In [ ]:
!pip install swig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.8 MB/s eta 0:00:00


In [ ]:
!pip install stable-baselines3[extra]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.7 MB/s eta 0:00:00


In [ ]:
!pip install gymnasium["all"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.4/374.4 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 16.7 MB/s eta 0:00:00
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp310-cp310-linux_x86_64.whl size=2376473 sha256=072aa29a67802a15d00c8bd5f2490995bf1cac3f10f4f1454bf2838844bb7a3f
  Stored in directory: /root/.cache/pip/wheels/db/8f/6a/eaaadf056fba10a98d986f6dce954e6201ba3126926fc5ad9e
Successfully built box2d-py
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.11
    Uninstalling Cython-3.0.11:
      Successfully uninstalled Cython-3.0.11


In [ ]:
import gymnasium as gym
from stable_baselines3 import PPO, A2C, DDPG
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
import torch as th
from torch import nn
import numpy as np

In [ ]:
#hyperparams
gamma = 0.5
lr = 0.001

In [ ]:
#for recording loss and reward
class PlotCallback(BaseCallback):
    def __init__(self, verbose=0):
        super().__init__(verbose)
        self.rewards = []
        self.losses = []

    def _on_step(self) -> bool:
        #save loss
        if hasattr(self.model, 'logger'):
            loss = self.model.logger.name_to_value.get('train/loss', np.nan)
            if not np.isnan(loss):
                self.losses.append(loss)
        return True

    def _on_rollout_end(self) -> None:
        #log rewards and calculate returns at the end of each episode
        if hasattr(self.model, 'ep_info_buffer') and self.model.ep_info_buffer:
            for ep_info in self.model.ep_info_buffer:
              self.rewards.append(ep_info['r'])

        self.model.ep_info_buffer.clear()

In [ ]:
#env
env = gym.make("Pusher-v5")

In [ ]:
#declare model
model = PPO("MlpPolicy", env, verbose=1, gamma=gamma, learning_rate=lr)

In [ ]:
#training
callback = PlotCallback()
training_history = model.learn(total_timesteps=200000, callback=callback, log_interval=1000)

In [ ]:
#plots
from matplotlib import pyplot as plt

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter( range(len(callback.rewards)), callback.rewards)
plt.xlabel("Episode")
plt.ylabel("Total Reward")
plt.title(f"gamma {gamma} lr {lr}")

plt.legend()

plt.subplot(1, 2, 2)
plt.plot(callback.losses, label="MSE (Loss)")
plt.xlabel("Training Steps")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
#model save
model.save("ppo_pusher")